In [23]:
tk = None

In [24]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#LSTM Training, Test
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings(action='ignore')

In [25]:
#불용어 사전 제작
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()

In [26]:
# data preprocessing
def preprocessing(data):
# 데이터 중복 값 확인 및 제거
# document: 약 4,000개 중복(총150,000개 - 146,182개), 
# label 2값 확인(0, 1만 가지기 때문에)
    
    # 중복 여부 검사, nunique()는 중복 제외하고 표시. list와 유사
    data['document'].nunique(), data['label'].nunique()
    
    # document 컬럼에서 중복인 내용이 있으면 중복 제거
    data.drop_duplicates(subset=['document'], inplace=True)
    data = data.dropna(how='any') # Null값이 존재하는 행 제거
    
    # 한글과 공백을 제외하고 모두 제거하는 정규 표현식 사용
    data['document'] = data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
    
    # 다시한번 Null값이 존재하는지 확인
    #공백만 있거나 빈 값을 가진 행이 있으면 Null로 변경
    data['document'] = data['document'].str.replace('^ +', "") 
    data['document'].replace('', np.nan, inplace=True) #Null값으로 변환
    
    # 데이터의 null 행 제거
    data = data.dropna(how='any')
    
    return data

In [27]:
# train data tokenization
def train_tokenizer(data):
# 입력값: Null값이 제거된 데이터셋 / 출력값: 불용어를 제거한 후 Null값도 제거한 데이터셋
# 기능: 불용어 제거, 단어 빈도수가 2회 이하인 단어 수를 찾아내고 공백 제거(공백 제거 목적)
    
    X_data=[]
    for sentence in data['document']:
        # 형태소 분석기(Okt())에서 토큰화(한글은 띄어쓰기) 실행. 
        # stem=True로 일정 수준 정규화(동사,명사화)
        X_tmp = okt.morphs(sentence, stem=True)
        X_tmp = [word for word in X_tmp if not word in stopwords] # 불용어 사전에 없으면 리스트에 추가
        X_data.append(X_tmp)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_data)
    
    # 등장 빈도수가 3회 미만인 단어들이 이 데이터에서 얼만큼 비중을 차지하는지 확인
    threshold = 3 # 단어의 등장 빈도수 기준
    rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
    total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
    rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 빈도수 총 합
    total_cnt = len(tokenizer.word_index) #단어의 수
    
    # 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
    for key, value in tokenizer.word_counts.items():
        total_freq = total_freq + value

        # 단어 빈도수가 threshold보다 작을 경우
        if(value < threshold):
            rare_cnt = rare_cnt + 1
            rare_freq = rare_freq + value

    # 빈도수가 2회 이하인 단어들은 제외
    # 0번일 경우를 고려하여 크기는 +1을 해준다.
    voca_size = total_cnt - rare_cnt + 1
    
    # 토큰화(단어단위로 쪼갬)
    tokenizer = Tokenizer(voca_size)
    # 단어에 숫자(인덱스)를 부여 
    tokenizer.fit_on_texts(X_data)
    
    y_data = np.array(data['label'])
    global tk
    tk = tokenizer
    X_data, y_data = train_padding(X_data, y_data)
    
    return X_data, y_data, voca_size

In [28]:
# train data padding
def train_padding(X_data, y_data):
    
    # texts_to_sequences, 단어에 순번을 지정, 0 ~ 19,415번 단어가 있음
    X_data = tk.texts_to_sequences(X_data) 

    #empty samples 제거
    # 단어가 1개 미만의 값이 없는 데이터 제거
    drop_data = [index for index, sentence in enumerate(X_data) if len(sentence) < 1] 

    # 빈 샘플 제거
    X_data = np.delete(X_data, drop_data, axis=0) # X_data에서 drop_data을 사용해서 제거
    y_data = np.delete(y_data, drop_data, axis=0)
    
    # 전체 훈련 데이터중 94%가 길이가 30 이하이므로 모든 샘플의 길이를 30으로 조정
    X_data = pad_sequences(X_data, maxlen = 30)
    
    return X_data, y_data

In [29]:
# test data tokenization
def test_tokenizer(data): 
# 입력값: Null값이 제거된 데이터셋 / 출력값: 불용어를 제거한 후 Null값도 제거한 데이터셋
# 기능: 불용어 제거, 단어 빈도수가 2회 이하인 단어 수를 찾아내고 공백 제거(공백 제거 목적)

    X_data=[]
    for sentence in data['document']:
        # 형태소 분석기(Okt())에서 토큰화(한글은 띄어쓰기) 실행. stem=True로 일정 수준 정규화(동사,명사화)
        X_tmp = okt.morphs(sentence, stem=True)
        X_tmp = [word for word in X_tmp if not word in stopwords] # 불용어 사전에 없으면 리스트에 추가
        X_data.append(X_tmp)

    y_data = np.array(data['label'])

    X_data = test_padding(X_data)
    
    return X_data, y_data

In [30]:
# test data padding
def test_padding(X_data):

    # texts_to_sequences, 단어에 순번을 지정, 0 ~ 19,415번 단어가 있음
    X_data = tk.texts_to_sequences(X_data) 

    #전체 훈련 데이터중 94%가 길이가 30 이하이므로 모든 샘플의 길이를 30으로 조정
    X_data = pad_sequences(X_data, maxlen = 30)

    return X_data

In [37]:
# data modeling
def lstm_modeling(X_data, y_data, X_data2, y_data2, voca_size): #입력값: / 출력값: /함수의 기능:

    model = Sequential()
    # Embedding(number of samples, size of vector, input length) 
    model.add(Embedding(voca_size, 100)) # vocab_size 개수만큼, 벡터 사이즈 100차원
    model.add(LSTM(128)) # LSTM cell 개수
    
    # 출력층 뉴런 1개, 활성화 함수 sigmoid(binary classification이기 때문)
    model.add(Dense(1, activation='sigmoid'))

    # 검증 데이터 손실이 증가하면, 과적합 위험. 검증 데이터 손실이 4회 증가하면 학습을 조기 종료
    # ModelCheckpoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델 저장
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

    # RMSProp, 세밀하게 학습하지만 상황에 따라 정도를 정하기 위함
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X_data, y_data, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

    loaded_model = load_model('best_model.h5')
    print("\n 테스트 정확도 : %0.4f" % (loaded_model.evaluate(X_data2, y_data2)[1]))

In [32]:
# test data prediction
def predict(new_sentence):
    loaded_model = load_model('best_model.h5')
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 
    pad_new = test_padding([new_sentence])
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [33]:
train_data = pd.read_table('C:/Users/CJ/Documents/project2/ratings_train.txt')
test_data = pd.read_table('C:/Users/CJ/Documents/project2/ratings_test.txt')

In [34]:
# data preprocessing
train_data = preprocessing(train_data)
test_data = preprocessing(test_data)

In [35]:
# data tokenization
X_train, y_train, voca_size = train_tokenizer(train_data)
X_test, y_test = test_tokenizer(test_data)

In [38]:
# lstm_modeling(X_train, y_train, voca_size)
lstm_modeling(X_train, y_train, X_test, y_test, voca_size)

Epoch 1/15
1934/1936 [============================>.] - ETA: 0s - loss: 0.3892 - acc: 0.8238
Epoch 00001: val_acc improved from -inf to 0.84394, saving model to best_model.h5
1936/1936 [==============================] - 67s 35ms/step - loss: 0.3892 - acc: 0.8238 - val_loss: 0.3535 - val_acc: 0.8439
Epoch 2/15
1935/1936 [============================>.] - ETA: 0s - loss: 0.3272 - acc: 0.8579
Epoch 00002: val_acc improved from 0.84394 to 0.85585, saving model to best_model.h5
1936/1936 [==============================] - 67s 35ms/step - loss: 0.3272 - acc: 0.8579 - val_loss: 0.3367 - val_acc: 0.8559
Epoch 3/15
1935/1936 [============================>.] - ETA: 0s - loss: 0.3026 - acc: 0.8717
Epoch 00003: val_acc improved from 0.85585 to 0.86002, saving model to best_model.h5
1936/1936 [==============================] - 60s 31ms/step - loss: 0.3025 - acc: 0.8717 - val_loss: 0.3296 - val_acc: 0.8600
Epoch 4/15
1936/1936 [==============================] - ETA: 0s - loss: 0.2834 - acc: 0.8823
E

In [43]:
predict('재밌어요')

94.95% 확률로 긍정 리뷰입니다.



In [44]:
predict('한번쯤은 볼만한 영화')

66.02% 확률로 긍정 리뷰입니다.



In [45]:
predict('쓰레기')

98.40% 확률로 부정 리뷰입니다.



In [46]:
predict('보라고 하고싶진 않지만 한번은 보세요')

55.14% 확률로 부정 리뷰입니다.



In [47]:
predict('꼭 보고싶다면 말리진 않을게요')

84.58% 확률로 긍정 리뷰입니다.



In [48]:
predict('이걸 돈주고 본다고?')

96.33% 확률로 부정 리뷰입니다.

